In [40]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import cross_val_score
import matplotlib.pyplot as plt

%matplotlib inline

In [20]:
train_df = pd.read_csv("data/dui.csv")
test_df = pd.read_csv("data/dui-test.csv")

In [21]:
train_df.head(10)

,Age,Gender,BAC,speeding5MPH,speeding10MPH,Fatality
0,56,M,0.017,0,0,0
1,38,M,0.034,0,0,1
2,22,F,0.000,1,0,0
3,71,F,0.046,1,1,0
4,63,F,0.000,0,0,0
5,90,M,0.035,0,0,3
6,73,F,0.036,0,0,0
7,15,F,0.103,1,0,0
8,88,F,0.042,1,0,0
9,51,M,0.066,1,1,2


In [22]:
train_df.loc[train_df['Fatality'] > 0, 'Fatality'] = 1

In [23]:
def clean_data(df, train=False):  
    le = LabelEncoder()
    df["Gender"] = le.fit_transform(df["Gender"])
    if train:
        df.loc[df['Fatality'] > 0, 'Fatality'] = 1
        df.loc[df['Fatality'] <= 0, 'Fatality'] = 0
    return df

In [29]:
train_df = clean_data(train_df, True)
test_df = clean_data(test_df, False)
train_X = train_df[["Age", "Gender", "BAC", "speeding5MPH", "speeding10MPH"]]
train_Y = train_df["Fatality"]
test_X = test_df[["Age", "Gender", "BAC", "speeding5MPH", "speeding10MPH"]]

In [30]:
train_df.head()

,Age,Gender,BAC,speeding5MPH,speeding10MPH,Fatality
0,56,1,0.017,0,0,0
1,38,1,0.034,0,0,1
2,22,0,0.000,1,0,0
3,71,0,0.046,1,1,0
4,63,0,0.000,0,0,0


In [33]:
clf = GradientBoostingClassifier()
score = cross_val_score(clf, train_X, train_Y, cv=5)
print(score)

[0.89362981 0.88942308 0.90384615 0.88822115 0.89116055]


In [39]:
learning_rate = [0.1, 0.01, 0.001, 0.0001]
n_estimators = [50, `100, 150, 200]
parameter_candidates = {'lr': learning_rate,
                        'n_estimators': n_estimators}
  {'C': [1, 10, 100, 1000], 'kernel': ['linear']},
  {'C': [1, 10, 100, 1000], 'gamma': [0.001, 0.0001], 'kernel': ['rbf']},
]

NameError: name 'roc_curve' is not defined